In [1]:
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer

from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM
import torch


model_pipeline = pipeline(
    model = "Qwen/Qwen2.5-3B-Instruct",
     device = "cuda"
)

result = model_pipeline("What is the preferred HIV ART second line?")
print(result)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


[{'generated_text': 'What is the preferred HIV ART second line? A: Tenofovir B: Stavudine C: Lamivudine D: Zidovudine\nThe preferred second-line antiretroviral therapy (ART) for individuals who are not able to tolerate or respond adequately to first-line regimens, such as those containing zidovudine (ZDV), is tenofovir.\n\nTherefore, the correct answer is:\n\nA: Tenofovir\n\nTenofovir is often recommended as a second-line option because it has fewer drug interactions and can be used in combination with other drugs that are less well tolerated by some patients. It also has a lower potential for renal toxicity compared to stavudine and lamivudine. \n\nTo summarize:\n- **First-line** options might include combinations like zidovudine, lamivudine, and efavirenz.\n- **Second-line** options typically include tenofovir-based regimens or other alternatives that provide an improved regimen based on patient tolerance and resistance patterns.\n\nThus, among the given options, tenofovir is the mos

In [2]:
print(result[0]['generated_text'])

What is the preferred HIV ART second line? A: Tenofovir B: Stavudine C: Lamivudine D: Zidovudine
The preferred second-line antiretroviral therapy (ART) for individuals who are not able to tolerate or respond adequately to first-line regimens, such as those containing zidovudine (ZDV), is tenofovir.

Therefore, the correct answer is:

A: Tenofovir

Tenofovir is often recommended as a second-line option because it has fewer drug interactions and can be used in combination with other drugs that are less well tolerated by some patients. It also has a lower potential for renal toxicity compared to stavudine and lamivudine. 

To summarize:
- **First-line** options might include combinations like zidovudine, lamivudine, and efavirenz.
- **Second-line** options typically include tenofovir-based regimens or other alternatives that provide an improved regimen based on patient tolerance and resistance patterns.

Thus, among the given options, tenofovir is the most preferred second-line ART agent.

In [3]:
del model_pipeline
del result

torch.cuda.empty_cache()


In [4]:


tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct"
)


In [5]:
train_data = load_dataset("json", data_files="training_data.json")


In [6]:
def preprocess(sample):

    sample = sample["prompt"]+"\n"+sample["completion"]
    tokenized = tokenizer(
            sample,
            max_length=128,
            truncation=True,
            padding="max_length"
        )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

In [7]:
data = train_data.map(preprocess)

In [11]:
tokenizer.save_pretrained("./guide_summary_qwen")
del tokenizer
torch.cuda.empty_cache()

In [12]:
print(data['train'][10])

{'prompt': 'Can rifepentine/Isoniazid be used for TB preventive therapy?', 'completion': '3 months of Rifapentine/Isoniazid (3HP) is the recommended TPT option except in pregnant and lactating women and children.', 'input_ids': [6713, 36924, 747, 26872, 38151, 930, 26975, 307, 387, 1483, 369, 30080, 70023, 15069, 5267, 18, 3951, 315, 89731, 391, 26872, 38151, 930, 26975, 307, 320, 18, 6610, 8, 374, 279, 11102, 350, 2828, 2999, 3650, 304, 20280, 323, 50544, 1095, 3198, 323, 2841, 13, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    device_map = "cuda",
    torch_dtype = torch.float16
)

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules = ["q_proj", "k_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
from transformers import TrainingArguments, Trainer
train_args = TrainingArguments(
    num_train_epochs = 15,
    learning_rate = 0.001,
    logging_steps = 25,
    fp16 = True,
    report_to="none"

)

In [15]:
trainer = Trainer(
    args=train_args,
    model=model,
    train_dataset=data["train"]
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [16]:
trainer.train()

Step,Training Loss
25,3.232300
50,0.964200
75,0.734600
100,0.485200
125,0.284100


TrainOutput(global_step=135, training_loss=1.0707113495579472, metrics={'train_runtime': 115.0742, 'train_samples_per_second': 9.385, 'train_steps_per_second': 1.173, 'total_flos': 2303587547873280.0, 'train_loss': 1.0707113495579472, 'epoch': 15.0})

In [18]:
trainer.save_model("./guide_summary_qwen")
# tokenizer.save_pretrained("./guide_summary_qwen")

In [19]:
from transformers import pipeline

ask_summary = pipeline(
    model="./guide_summary_qwen",
    tokenizer="./guide_summary_qwen",
    device="cuda"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


In [20]:
ask_summary("What is the preferred HIV ART second line?")[0]["generated_text"]

'What is the preferred HIV ART second line? Second-line ART regimens should be selected according to local ART resistance data and viral load results. Regimens should be selected if they are <  50 copies/mL  and the patient has a second-line indication based on clinical or immunological criteria. Patients with a second-line viral load result that is negative for HIV resistance should be re-evaluated in 6 months.'